# PIPELINE OF FEATURE EXTRACTION

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from skimage import exposure
from scipy import ndimage
from cellpose import models
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage import filters
from skimage import morphology
from skimage import measure
from matplotlib import colors
from importlib import reload
import utility
reload(utility)
from utility import get_hard_disk_path, compute_big_cell_labels, compute_small_cell_labels, calculate_closest_boundary_distances

initial_path = r"E:\New_data_Maxim\croped_images\\"
all_labels = ["keep2", "reseed1", "split", "keep0", "keep1", "reseed0"]
labels_big_cell = ["keep2", "reseed1", "split", "keep0", "keep1", "reseed0"]
labels_small_cell = []
properties_list = ['area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity']
output_csv = initial_path + "statistics_features_IP.csv"  # Adjust the output file name

def compute_feature_statistics(data, output_csv, properties_list):
    statistics = {}

    other_features = {
        "image_name": data["image_name"].iloc[0],  # Assuming 'image_name' is the same for all rows in data
        "label": data["label"].iloc[0],  # Assuming 'label' is the same for all rows in data
        "count": data["sample"].max(),
        "occupancy": data["occupancy"],
        #"density": data["density"]
    }
    statistics.update(other_features)

    for column_name in properties_list:
        if column_name in data.columns:
            column = data[column_name]

            # Calculate statistics for the current column
            mean_value = column.mean()
            sd_value = column.std()
            min_value = column.min()
            max_value = column.max()

            # Add the statistics to the row dictionary
            statistics.update({
                f"{column_name}_Mean": mean_value,
                f"{column_name}_StdDev": sd_value,
                f"{column_name}_Min": min_value,
                f"{column_name}_Max": max_value,
            })

    # Create a DataFrame from the dictionary
    statistics_df = pd.DataFrame(statistics, index=[0])

    # Save the DataFrame to the output CSV file
    statistics_df.to_csv(output_csv, mode='a', header=not os.path.isfile(output_csv), index=False)

def compute_occupancy(labels, image):
    """
    Compute the occupancy of cells in the image.
    
    Parameters:
    - labels: labeled image
    - image: original grayscale image
    
    Returns:
    - occupancy: ratio of cell area to total image area
    """
    cell_area = np.sum(labels > 0)  # Total number of pixels occupied by cells
    image_area = image.size         # Total number of pixels in the image
    return cell_area / image_area

if os.path.exists(output_csv):
    os.remove(output_csv)

if not os.path.exists(initial_path + "labeled/"):
    os.makedirs(initial_path + "labeled/")

for label in all_labels:
    label_path = os.path.join(initial_path + "labeled", label)
    if not os.path.exists(label_path):
        os.mkdir(label_path)
    if not os.path.exists(os.path.join(label_path, "IP_pipeline")):
        os.mkdir(os.path.join(label_path, "IP_pipeline"))
    os.chdir(os.path.join(initial_path, label))
    files = [file for file in os.listdir() if "Simple" not in file and "filled" not in file]
    print(files)
    for file in files:
        if "Simple" or "filled" not in file:
            image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
            if label in labels_big_cell:
                print(file)
                labels = compute_big_cell_labels(image)
            elif label in labels_small_cell:
                labels = compute_small_cell_labels(image)
            # Generate a colormap for labels
            label_cmap = colors.ListedColormap(np.random.rand(labels.max() + 1, 3))

            # Create a colored label image
            colored_label_image = label_cmap(labels)

            # Convert to uint8 format (0-255)
            colored_label_image = (colored_label_image * 255).astype(np.uint8)

            output_path = os.path.join(label_path, "IP_pipeline", file)
            if not os.path.exists(output_path):
                cv2.imwrite(output_path, cv2.cvtColor(colored_label_image, cv2.COLOR_RGBA2BGRA))

            info_table = pd.DataFrame(
                measure.regionprops_table(
                    labels,
                    intensity_image=image,
                    properties=['label', 'area', 'mean_intensity', 'solidity', 'convex_area', 'equivalent_diameter', 'perimeter', 'extent', 'max_intensity', 'min_intensity', 'eccentricity'],
                )
            )

            occupancy = compute_occupancy(labels, image)
            #density = calculate_closest_boundary_distances(labels)
            #info_table['density'] = density
            info_table['occupancy'] = occupancy  # Add the occupancy to the statistics
            info_table = info_table.rename(columns={"label": "sample"})
            info_table['label'] = label
            info_table['image_name'] = file

            compute_feature_statistics(info_table, output_csv, properties_list)

['2023_12_14_10h00_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene42.png', '2023_12_14_10h00_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene43.png', '2023_12_14_10h00_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene45.png', '2023_12_14_10h00_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene48.png', '2023_12_14_10h00_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene49.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene42.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene43.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene45.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene48.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene49.png', '2023_12_15_06h54_Plate#1_time_point-01_AcquisitionBlock4_pt4_10x_stitched_Scene56.png', '2023_12_15_06h54_Pl